# Install

In [ ]:
pip install torch

In [ ]:
pip install torchvision

# Imports and define NN

In [1]:
import ray
from ray.train.torch import TorchTrainer
from torchvision import datasets, transforms
import os
import tempfile
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(1, 32)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(32, 1)

    def forward(self, input):
        return self.layer2(self.relu(self.layer1(input)))

# Define Training Loop

In [2]:
# Training loop.
def train_loop_per_worker(config):

    # Read configurations.
    lr = config["lr"]
    batch_size = config["batch_size"]
    num_epochs = config["num_epochs"]

    # Fetch training dataset.
    train_dataset_shard = ray.train.get_dataset_shard("train")

    # Instantiate and prepare model for training.
    model = NeuralNetwork()
    model = ray.train.torch.prepare_model(model)

    # Define loss and optimizer.
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    # Create data loader.
    dataloader = train_dataset_shard.iter_torch_batches(
        batch_size=batch_size, dtypes=torch.float
    )

    # Train multiple epochs.
    for epoch in range(num_epochs):

        # Train epoch.
        for batch in dataloader:
            output = model(batch["input"])
            loss = loss_fn(output, batch["label"])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Create checkpoint.
        base_model = (model.module
            if isinstance(model, DistributedDataParallel) else model)
        checkpoint_dir = tempfile.mkdtemp()
        torch.save(
            {"model_state_dict": base_model.state_dict()},
            os.path.join(checkpoint_dir, "model.pt"),
        )
        checkpoint = Checkpoint.from_directory(checkpoint_dir)

        # Report metrics and checkpoint.
        ray.train.report({"loss": loss.item()}, checkpoint=checkpoint)

# Initializing the Cluster

In [3]:
# Initialize Ray
ray.init()

2024-07-11 23:15:52,313	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.14
Ray version:,2.10.0
Dashboard:,http://127.0.0.1:8265


(RayTrainWorker pid=2027) Setting up process group for: env:// [rank=0, world_size=4]
(TorchTrainer pid=2010) Started distributed worker processes: 
(TorchTrainer pid=2010) - (ip=127.0.0.1, pid=2027) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=2010) - (ip=127.0.0.1, pid=2028) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=2010) - (ip=127.0.0.1, pid=2029) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=2010) - (ip=127.0.0.1, pid=2030) world_rank=3, local_rank=3, node_rank=0
(RayTrainWorker pid=2027) Moving model to device: cpu
(RayTrainWorker pid=2027) Wrapping provided model in DistributedDataParallel.
(SplitCoordinator pid=2037) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(SplitCoordinator pid=2037) I0000 00:00:1720754211.920469 4292714 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new

In [5]:
from ray.train import Checkpoint, CheckpointConfig, RunConfig, ScalingConfig, FailureConfig
from ray.train.torch import TorchTrainer

max_failures = 1
num_workers = 4
use_gpu = False
train_loop_config = {"num_epochs": 20, "lr": 0.01, "batch_size": 32}
scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)
run_config = RunConfig(checkpoint_config=CheckpointConfig(num_to_keep=1), failure_config=FailureConfig(max_failures))

# Define datasets.
train_dataset = ray.data.from_items(
    [{"input": [x], "label": [2 * x + 1]} for x in range(2000)]
)
datasets = {"train": train_dataset}

# Initialize the Trainer.
trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config=train_loop_config,
    scaling_config=scaling_config,
    run_config=run_config,
    datasets=datasets,
)

# Running Training Job

In [6]:
# Start training
result = trainer.fit()
last_checkpoint = result.checkpoint

(pid=2037) - split(4, equal=True) 1:   0%|                                                                    …

(pid=2037) Running 0:   0%|                                                                                   …

(pid=2037) - split(4, equal=True) 1:   0%|                                                                    …

(pid=2037) Running 0:   0%|                                                                                   …

2024-07-11 23:16:52,311	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).


...

(pid=2037) Running 0:   0%|                                                                                   …

2024-07-11 23:16:55,271	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).


(pid=2037) - split(4, equal=True) 1:   0%|                                                                    …

(pid=2037) Running 0:   0%|                                                                                   …

2024-07-11 23:16:55,446	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-07-11 23:16:56,489	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.


In [8]:
result.metrics

{'loss': 5712591192064.0,
 'timestamp': 1720754215,
 'checkpoint_dir_name': 'checkpoint_000019',
 'should_checkpoint': True,
 'done': True,
 'training_iteration': 20,
 'trial_id': '2b370_00000',
 'date': '2024-07-11_23-16-55',
 'time_this_iter_s': 0.17356491088867188,
 'time_total_s': 7.395307779312134,
 'pid': 2010,
 'hostname': 'farceo-mac',
 'node_ip': '127.0.0.1',
 'config': {'train_loop_config': {'num_epochs': 20,
   'lr': 0.01,
   'batch_size': 32}},
 'time_since_restore': 7.395307779312134,
 'iterations_since_restore': 20,
 'experiment_tag': '0'}

# Scoring new data

In [15]:
def score_model(checkpoint_path, data_points):
    model_state_dict = torch.load(checkpoint_path)["model_state_dict"]
    model = NeuralNetwork()
    model.load_state_dict(model_state_dict)
    model.eval()  # Set the model to evaluation mode
    
    with torch.no_grad():  # Disable gradient calculation for inference
        inputs = torch.tensor([data_points], dtype=torch.float).reshape(-1, 1)
        predictions = model(inputs)
        return predictions.numpy()

In [28]:
checkpoint_path = open(os.path.join(last_checkpoint.path, "model.pt"), 'rb')

predictions = score_model(checkpoint_path, train_dataset.to_pandas()['input'].values)

/var/folders/v0/fgw7yz9j5ml1vxvq6j0rqf9w0000gn/T/ipykernel_1453/354454316.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  inputs = torch.tensor([data_points], dtype=torch.float).reshape(-1, 1)


In [29]:
predictions

array([[-2338410.],
       [-2338410.],
       [-2338410.],
       ...,
       [-2338410.],
       [-2338410.],
       [-2338410.]], dtype=float32)

# End